## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

ALG_DATA_PATH = os.path.join(DATA_PATH, "latticized")
try_mkdir(ALG_DATA_PATH)

In [3]:
VISUALIZE = False
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 1
CLEARANCE = 1e-3

ROBOT_TYPE = RobotType.indy7
ROBOT_NAME = "indy0"
TOOL_LINK = "indy0_tcp"
TOOL_XYZ = (0,0,0.14)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.05
HOME_POSE = (0,0,0,0,0,0)

# ROBOT_TYPE = RobotType.panda
# ROBOT_NAME = "panda0"
# TOOL_LINK = "panda0_hand"
# TOOL_XYZ = (0,0,0.112)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.03
# HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan = MoveitPlanner(pscene)
checker = MoveitPlanner(pscene)

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checker.motion_filters = [gcheck] # rchecker is not perfect, don't use for data gen

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Object Classes

In [13]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

##
# @class ObstacleBase
# @brief base class for obstacle generators
class ObstacleBase:
    RTH_MIN = None ## R: center ~ nearest point
    RTH_MAX = None
    RPY_MIN = None
    RPY_MAX = None
    DIM_MIN = None
    DIM_MAX = None
    GTYPE = None
    COLOR = (0.7,0.7,0.7,1)
    
    def __init__(self, gscene, name, sampler=np.random.uniform, DIM=None, RTH=None, RPY=None):
        self.name = name
        self.DIM = sampler(self.DIM_MIN, self.DIM_MAX) if DIM is None else DIM
        self.RTH = sampler(self.RTH_MIN, self.RTH_MAX) if RTH is None else RTH
        self.RPY = sampler(self.RPY_MIN, self.RPY_MAX) if RPY is None else RPY
        self.RPY[2] += self.RTH[1]
        self.XYZ = np.array(cyl2cart(*self.RTH))
        verts_rotated = np.matmul(Rot_rpy(self.RPY), (DEFAULT_VERT_DICT[self.GTYPE]*self.DIM).transpose())
        xy_normed = self.XYZ[:2]/(np.linalg.norm(self.XYZ[:2])+1e-6)
        verts_r_compo = np.dot(xy_normed, verts_rotated[:2,:])
        self.XYZ[:2] -= xy_normed[:2]*np.min(verts_r_compo)
        self.RTH[0] -= np.min(verts_r_compo)
        self.geometry = gscene.create_safe(gtype=self.GTYPE, name=self.name, link_name="base_link", 
                                  dims=self.DIM, center=tuple(self.XYZ), rpy=self.RPY, 
                                  color=self.COLOR, display=True, collision=True, fixed=True)
        self.subgeo_list = []
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_me, raddii_me = self.geometry.get_vertice_radius()
        verts_me_global = np.add(np.matmul(verts_me, self.geometry.orientation_mat.transpose()), 
                                 self.geometry.center)
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_me_global))-radii-raddii_me < 1e-4
        
##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.8, 1.5, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,0.5)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.3
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
        
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, -np.pi/2)
    RPY_MAX = (0, 0, +np.pi/2)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, +0.0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Ceiling
# @brief Ceiling - upperbound of the workspace
class Ceiling(ObstacleBase):
    RTH_MIN = (0.0, 0, 1.0)
    RTH_MAX = (0.0, 0, 2)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Wall
# @brief define horizontal boundary of the workspace
class Wall(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (2.0, np.pi, 0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 6, 3)
    DIM_MAX = (0.1, 6, 3)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.8, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.3, 0.3, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class Bar
# @brief occasional poles that obstruct robot motion
class Bar(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.8, +np.pi, 1.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 4, 0.1)
    DIM_MAX = (0.3, 4, 0.3)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
    DIM_MAX = (0.06, 0.3, 0.3)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.2,0.8,0.5)
    def __init__(self, gscene, name, workplane, XYZ_LOC=None, **kwargs):
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        
def clear_class(gscene, key, Nmax):
    for iw in range(Nmax):
        gname = "{}_{}".format(key, iw)
        if gname in gscene.NAME_DICT:
            gscene.remove(gscene.NAME_DICT[gname])

            
def redistribute_class(gscene, obstacle_class, key, Nmax, workplane_avoid=None):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    for iw in range(np.random.choice(Nmax)):
        obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        while workplane_avoid is not None and workplane_avoid.is_overlapped_with(obs.geometry):
            obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        obs_list.append(obs)
    return obs_list

            
def disperse_objects(gscene, object_class, key, Nmax, workplane_on):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    for iw in range(np.random.choice(Nmax)+1):
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0.3)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_a", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_b", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,-np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,-np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,+np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2], [0,0,0])}
    action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles
        
WORKPLANE_TYPES = [WorkPlane, Box, SideBox, TopBox]

Nmax_wall, Nmax_pole, Nmax_bar = 4, 8, 8

Nmax_obj = 3

In [14]:
from pkg.planning.filtering.lattice_model.latticizer_py import *
import itertools

ltc_full = Latticizer_py(WDH=(3, 3, 3), L_CELL=0.05, OFFSET_ZERO=(1.5, 1.5, 1.5))
ltc_effector = Latticizer_py(WDH=(1, 1, 1), L_CELL=0.05, OFFSET_ZERO=(0.5, 0.5, 0.5))
ltc_arm_05 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.05, OFFSET_ZERO=(0.5, 1.0, 1.0))
ltc_arm_10 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.10, OFFSET_ZERO=(0.5, 1.0, 1.0))

### sampling

In [15]:
gtimer = GlobalTimer.instance()
gtimer.reset()

reach_success_list = []
reach_time_list = []
retrieve_success_list = []
retrieve_time_list = []

for _ in range(9):
    ROBOT_DATA_ROOT = os.path.join(ALG_DATA_PATH, ROBOT_TYPE.name+"-failmore")
    try_mkdir(ROBOT_DATA_ROOT)

    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
    try_mkdir(DATASET_PATH)
    print("")
    print("DATASET_PATH: {}".format(DATASET_PATH))
    print("")

    GRASP_PATH = os.path.join(DATASET_PATH, "grasp")
    try_mkdir(GRASP_PATH)

    ARM_10_PATH = os.path.join(DATASET_PATH, "arm_10")
    try_mkdir(ARM_10_PATH)

    ARM_05_PATH = os.path.join(DATASET_PATH, "arm_05")
    try_mkdir(ARM_05_PATH)

    FULL_SCENE_PATH = os.path.join(DATASET_PATH, "full_scene")
    try_mkdir(FULL_SCENE_PATH)

    N_max_sample = 1000
    N_print = 5

    gtimer.tic("full_loop")

    i_s = 0
    i_print = 0
    reach_list = []
    retrieve_list = []
    while i_s < N_max_sample:
        ## add floor, ceiling
        floor = Floor(gscene, "floor")
        # ceiling = Ceiling(gscene, "ceiling")

        ## set workplane
        wp = random.choice(WORKPLANE_TYPES)(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

        ## add walls, poles, bars
        walls = redistribute_class(gscene, Wall, "wl", Nmax_wall, workplane_avoid=wp)
        poles = redistribute_class(gscene, Pole, "po", Nmax_pole, workplane_avoid=wp)
        bars = redistribute_class(gscene, Bar, "bar", Nmax_bar, workplane_avoid=wp)

        ## add object
        obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

        samples = []
        for obj in obj_list:
            SHOW_PERIOD = 0.01
            N_sample_single_obj = 3
            N_sample_single_obj_max = 3
            obj_pscene, handles = add_object(pscene, obj)
            mplan.update_gscene()
            initial_state = pscene.initialize_state(HOME_POSE)
            pscene.set_object_state(initial_state)
            from_state = initial_state.copy(pscene)
            to_node = ("grip0",)
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node, HOME_DICT)
            samples_obj = []
            for _ in range(N_sample_single_obj):
                to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
                
                pscene.set_object_state(from_state)
                
                Traj_self, LastQ, error, res, binding_list = \
                            checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                                    only_self_collision=True, timeout=TIMEOUT_SELF)
                if VISUALIZE and res:
                    gscene.show_motion(Traj_self)

                success_reach, success_retrieve = False, False
                time_reach, time_retrieve = TIMEOUT_REACH, TIMEOUT_RETRIEVE
                if res:
                    gtimer.tic("reach")
                    Traj_reach, LastQ, error, success_reach, binding_list = mplan.plan_transition(
                        from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, timeout=TIMEOUT_REACH)
                    time_reach = gtimer.toc("reach")
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
        #             print("reach: {}".format(success_reach))
                    Traj_retrieve = []
                    if success_reach:
                        if VISUALIZE:
                            gscene.show_motion(Traj_reach, period=SHOW_PERIOD)
                        for bd in binding_list:
                            pscene.rebind(bd, list2dict(LastQ, pscene.gscene.joint_names))
                        binding_state, state_param = pscene.get_object_state()
                        new_state = State(binding_state, state_param, list(LastQ), pscene)
                        end_state = new_state.copy(pscene)
                        end_state.Q = np.array(HOME_POSE)
                        gtimer.tic("retrieve")
                        Traj_retrieve, LastQ, error, success_retrieve, binding_list = mplan.plan_transition(
                            from_state=new_state, to_state=end_state, timeout=TIMEOUT_RETRIEVE)
                        time_retrieve = gtimer.toc("retrieve")
                        retrieve_success_list.append(success_retrieve)
                        retrieve_time_list.append(time_retrieve)
        #                 print("retrieve: {}".format(success_retrieve))
                        if success_retrieve and VISUALIZE:
                            gscene.show_motion(Traj_retrieve, period=SHOW_PERIOD)
                    samples_obj.append((obj.name, from_state, to_state, redundancy_dict, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
                    if len(samples_obj)>=N_sample_single_obj_max:
                        pscene.set_object_state(initial_state)
                        break
                else:
                    Traj_reach, Traj_retrieve = [], []
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
                    retrieve_success_list.append(success_retrieve)
                    retrieve_time_list.append(time_retrieve)
                    samples_obj.append((obj.name, from_state, to_state, redundancy_dict, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
#                 if success_retrieve and not res_col:
#                     raise(RuntimeError("Something's wrong: successful motion but collision checker failed"))
        #             print("no save?")
            samples = samples+samples_obj
            pscene.set_object_state(initial_state)

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)    

            gscene.update_markers_all()

        for sample in samples:
            obj_name, from_state, to_state, redundancy_dict, success_reach, success_retrieve, \
                Traj_reach, Traj_retrieve, time_reach, time_retrieve = sample
            obj = [obj for obj in obj_list if obj.name == obj_name][0]
            obj_pscene, handles = add_object(pscene, obj)
            pscene.set_object_state(from_state)
            binding_list, binding_ok = pscene.get_slack_bindings(from_state, to_state)
            assert binding_ok, "no available transition"
            assert len(binding_list) == 1, "multiple binding transition - only single allowed in training process"
            binding = binding_list[0]

            grasp_dict = {}
            arm_05_dict = {}
            arm_10_dict = {}
            full_scene_dict = {}
            ltc_effector.clear()
            ltc_arm_05.clear()
            ltc_arm_10.clear()
            ltc_full.clear()

            obj_name, ap_name, binder_name, binder_geometry_name = binding
            actor, obj = pscene.actor_dict[binder_name], pscene.subject_dict[obj_name]
            handle = obj.action_points_dict[ap_name]
            redundancy, Q_dict = redundancy_dict[obj_name], list2dict(from_state.Q, gscene.joint_names)
            redundancy = redundancy_dict[obj_name]
            point_add_handle, rpy_add_handle = calc_redundancy(redundancy[ap_name], handle)
            point_add_actor, rpy_add_actor = calc_redundancy(redundancy[binder_name], actor)
            
            T_handle_lh = np.matmul(handle.Toff_lh, SE3(Rot_rpy(rpy_add_handle), point_add_handle))
            T_actor_lh = np.matmul(actor.Toff_lh, SE3(Rot_rpy(rpy_add_actor), point_add_actor))
            T_loal = np.matmul(T_handle_lh, SE3_inv(T_actor_lh))
            actor_vertinfo_list, object_vertinfo_list, actor_Tinv_dict, object_Tinv_dict = \
                                                gcheck.get_grasping_vert_infos(actor, obj, T_loal, HOME_DICT)

            if all([ROBOT_NAME in lname for lname in actor_Tinv_dict.keys()]):
                tool_Tinv_dict = actor_Tinv_dict
                tool_vertinfo_list = actor_vertinfo_list
                target_vertinfo_list = object_vertinfo_list
                T_end_effector = T_loal
            elif all([ROBOT_NAME in lname for lname in object_Tinv_dict.keys()]):
                tool_Tinv_dict = object_Tinv_dict
                tool_vertinfo_list = object_vertinfo_list
                target_vertinfo_list = actor_vertinfo_list
                T_end_effector = SE3_inv(T_loal)
            else:
                raise("Invaild TOOL_LINK_BUNDLE")
            T_end_joint = T_end_effector
#             for lname in TOOL_LINK_BUNDLE:
#                 T_end_joint = np.matmul(T_end_joint, tool_Tinv_dict[lname])
#             raise

            r, th, h = cart2cyl(*T_end_effector[:3,3])
            Tref = SE3(Rot_axis(3, th), T_end_effector[:3,3])
            obj_names = [obj.geometry.name] + obj.geometry.children
            target_names= [item[0] for item in target_vertinfo_list if item[0] not in obj_names]
            tool_names = [item[0] for item in tool_vertinfo_list]

            ltc_effector.convert_vertices(tool_vertinfo_list, Tref=Tref)
            ltc_effector.convert_vertices(target_vertinfo_list, Tref=Tref)

            Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
            ltc_arm_05.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_full.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_full.convert_vertices(actor_vertinfo_list, Tref=Tref_base)

            grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_end_effector, T_end_joint, Tref_base
            grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
            grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

            arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

            arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

            full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

            reach_list.append(success_reach)
            retrieve_list.append(success_retrieve)

            save_pickle(os.path.join(GRASP_PATH, "%06d.pkl"%i_s), grasp_dict)
            save_pickle(os.path.join(ARM_05_PATH, "%06d.pkl"%i_s), arm_05_dict)
            save_pickle(os.path.join(ARM_10_PATH, "%06d.pkl"%i_s), arm_10_dict)
            save_pickle(os.path.join(FULL_SCENE_PATH, "%06d.pkl"%i_s), full_scene_dict)

            i_s += 1

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)

            gscene.update_markers_all()
        if i_s > 0 :
            if i_s > i_print*N_print:
                i_print +=  1
                print_end = "\n"
            else:
                print_end = "\r"
            time_elapsed = gtimer.toc("full_loop")/1000
            print("{} / {} in {} / {} s -- reach,retrieve = ({} %, {} %)                     ".format(
                i_s, N_max_sample, round(time_elapsed, 2), round(time_elapsed/i_s*N_max_sample, 2), 
                round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)), end=print_end)


    print("")
    print("")
    print("============= Finished {} in {} s -- reach,retrieve = ({} %, {} %) =================".format(
        i_s, round(time_elapsed, 2), round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)))


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/latticized/indy7-failmore/20210609-031118

9 / 1000 in 9.77 / 1085.58 s -- reach,retrieve = (0.0 %, 0.0 %)                     
15 / 1000 in 12.95 / 863.53 s -- reach,retrieve = (0.0 %, 0.0 %)                     
21 / 1000 in 18.91 / 900.47 s -- reach,retrieve = (0.0 %, 0.0 %)                     
24 / 1000 in 20.13 / 838.84 s -- reach,retrieve = (0.0 %, 0.0 %)                     
30 / 1000 in 23.19 / 773.07 s -- reach,retrieve = (0.0 %, 0.0 %)                     
33 / 1000 in 25.38 / 769.24 s -- reach,retrieve = (0.0 %, 0.0 %)                     
42 / 1000 in 61.56 / 1465.83 s -- reach,retrieve = (2.4 %, 2.4 %)                     
45 / 1000 in 62.98 / 1399.48 s -- reach,retrieve = (2.2 %, 2.2 %)                     
48 / 1000 in 65.67 / 1368.09 s -- reach,retrieve = (2.1 %, 2.1 %)                     
57 / 1000 in 103.85 / 1821.89 s -- reach,retrieve = (1.8 %, 1.8 %)                     
63 / 1000 in 110.8 / 1758.71 s -- reach,r

501 / 1000 in 747.62 / 1492.26 s -- reach,retrieve = (5.4 %, 5.4 %)                     
507 / 1000 in 754.23 / 1487.63 s -- reach,retrieve = (5.3 %, 5.3 %)                     
513 / 1000 in 759.2 / 1479.93 s -- reach,retrieve = (5.3 %, 5.3 %)                     
516 / 1000 in 760.55 / 1473.93 s -- reach,retrieve = (5.2 %, 5.2 %)                     
519 / 1000 in 762.09 / 1468.38 s -- reach,retrieve = (5.2 %, 5.2 %)                     
522 / 1000 in 766.15 / 1467.73 s -- reach,retrieve = (5.2 %, 5.2 %)                     
525 / 1000 in 767.84 / 1462.56 s -- reach,retrieve = (5.1 %, 5.1 %)                     
528 / 1000 in 771.48 / 1461.14 s -- reach,retrieve = (5.1 %, 5.1 %)                     
531 / 1000 in 774.2 / 1458.0 s -- reach,retrieve = (5.1 %, 5.1 %)                     
534 / 1000 in 776.3 / 1453.74 s -- reach,retrieve = (5.1 %, 5.1 %)                     
543 / 1000 in 790.46 / 1455.72 s -- reach,retrieve = (5.0 %, 5.0 %)                     
546 / 1000 in 792.18 / 14

966 / 1000 in 1327.87 / 1374.61 s -- reach,retrieve = (4.5 %, 4.5 %)                     
969 / 1000 in 1330.44 / 1373.0 s -- reach,retrieve = (4.4 %, 4.4 %)                     
978 / 1000 in 1340.27 / 1370.42 s -- reach,retrieve = (4.4 %, 4.4 %)                     
984 / 1000 in 1346.93 / 1368.83 s -- reach,retrieve = (4.4 %, 4.4 %)                     
987 / 1000 in 1349.7 / 1367.48 s -- reach,retrieve = (4.4 %, 4.4 %)                     
993 / 1000 in 1354.36 / 1363.91 s -- reach,retrieve = (4.3 %, 4.3 %)                     
996 / 1000 in 1356.55 / 1361.99 s -- reach,retrieve = (4.3 %, 4.3 %)                     
999 / 1000 in 1359.64 / 1361.0 s -- reach,retrieve = (4.4 %, 4.4 %)                     
1005 / 1000 in 1365.52 / 1358.73 s -- reach,retrieve = (4.4 %, 4.4 %)                     


============= Finished 1005 in 1365.52 s -- reach,retrieve = (4.4 %, 4.4 %) =================

DATASET_PATH: /home/rnb/Projects/rnb-planning/data/latticized/indy7-failmore/20210609-033403

3

459 / 1000 in 466.32 / 1015.95 s -- reach,retrieve = (3.7 %, 3.5 %)                     
462 / 1000 in 469.15 / 1015.47 s -- reach,retrieve = (3.7 %, 3.5 %)                     
471 / 1000 in 475.1 / 1008.71 s -- reach,retrieve = (3.6 %, 3.4 %)                     
480 / 1000 in 481.8 / 1003.76 s -- reach,retrieve = (3.5 %, 3.3 %)                     
486 / 1000 in 486.62 / 1001.27 s -- reach,retrieve = (3.5 %, 3.3 %)                     
489 / 1000 in 519.07 / 1061.49 s -- reach,retrieve = (3.5 %, 3.3 %)                     
492 / 1000 in 520.95 / 1058.84 s -- reach,retrieve = (3.5 %, 3.3 %)                     
498 / 1000 in 525.66 / 1055.54 s -- reach,retrieve = (3.4 %, 3.2 %)                     
501 / 1000 in 527.85 / 1053.59 s -- reach,retrieve = (3.4 %, 3.2 %)                     
507 / 1000 in 533.05 / 1051.38 s -- reach,retrieve = (3.4 %, 3.2 %)                     
510 / 1000 in 567.04 / 1111.84 s -- reach,retrieve = (3.5 %, 3.3 %)                     
513 / 1000 in 568.78 / 

942 / 1000 in 1040.58 / 1104.65 s -- reach,retrieve = (3.7 %, 3.6 %)                     
948 / 1000 in 1043.49 / 1100.73 s -- reach,retrieve = (3.7 %, 3.6 %)                     
957 / 1000 in 1048.2 / 1095.3 s -- reach,retrieve = (3.9 %, 3.8 %)                     
963 / 1000 in 1052.18 / 1092.6 s -- reach,retrieve = (3.8 %, 3.7 %)                     
966 / 1000 in 1055.69 / 1092.85 s -- reach,retrieve = (3.8 %, 3.7 %)                     
972 / 1000 in 1057.43 / 1087.89 s -- reach,retrieve = (3.8 %, 3.7 %)                     
975 / 1000 in 1059.73 / 1086.9 s -- reach,retrieve = (3.8 %, 3.7 %)                     
981 / 1000 in 1065.53 / 1086.16 s -- reach,retrieve = (3.8 %, 3.7 %)                     
987 / 1000 in 1070.38 / 1084.48 s -- reach,retrieve = (3.7 %, 3.6 %)                     
990 / 1000 in 1073.42 / 1084.27 s -- reach,retrieve = (3.7 %, 3.6 %)                     
993 / 1000 in 1075.23 / 1082.81 s -- reach,retrieve = (3.7 %, 3.6 %)                     
999 / 1000 in 

...

645 / 1000 in 898.69 / 1393.32 s -- reach,retrieve = (5.7 %, 5.7 %)                     
651 / 1000 in 964.96 / 1482.27 s -- reach,retrieve = (5.7 %, 5.7 %)                     
657 / 1000 in 968.0 / 1473.36 s -- reach,retrieve = (5.6 %, 5.6 %)                     
663 / 1000 in 970.78 / 1464.22 s -- reach,retrieve = (5.6 %, 5.6 %)                     
669 / 1000 in 976.19 / 1459.18 s -- reach,retrieve = (5.5 %, 5.5 %)                     
672 / 1000 in 977.19 / 1454.15 s -- reach,retrieve = (5.5 %, 5.5 %)                     
678 / 1000 in 982.94 / 1449.76 s -- reach,retrieve = (5.5 %, 5.5 %)                     
681 / 1000 in 984.67 / 1445.92 s -- reach,retrieve = (5.4 %, 5.4 %)                     
690 / 1000 in 990.31 / 1435.22 s -- reach,retrieve = (5.4 %, 5.4 %)                     
696 / 1000 in 997.2 / 1432.76 s -- reach,retrieve = (5.3 %, 5.3 %)                     
702 / 1000 in 1002.78 / 1428.46 s -- reach,retrieve = (5.3 %, 5.3 %)                     
711 / 1000 in 1013.89 

138 / 1000 in 119.1 / 863.07 s -- reach,retrieve = (8.0 %, 8.0 %)                     
144 / 1000 in 122.26 / 849.02 s -- reach,retrieve = (7.6 %, 7.6 %)                     
150 / 1000 in 128.71 / 858.08 s -- reach,retrieve = (7.3 %, 7.3 %)                     
156 / 1000 in 136.72 / 876.44 s -- reach,retrieve = (7.1 %, 7.1 %)                     
159 / 1000 in 138.99 / 874.13 s -- reach,retrieve = (7.5 %, 7.5 %)                     
162 / 1000 in 140.61 / 867.98 s -- reach,retrieve = (7.4 %, 7.4 %)                     
168 / 1000 in 145.35 / 865.19 s -- reach,retrieve = (8.3 %, 8.3 %)                     
171 / 1000 in 146.66 / 857.64 s -- reach,retrieve = (8.8 %, 8.8 %)                     
177 / 1000 in 151.07 / 853.49 s -- reach,retrieve = (8.5 %, 8.5 %)                     
183 / 1000 in 156.79 / 856.78 s -- reach,retrieve = (8.2 %, 8.2 %)                     
189 / 1000 in 164.16 / 868.56 s -- reach,retrieve = (9.0 %, 9.0 %)                     
195 / 1000 in 167.48 / 858.88 s -

606 / 1000 in 692.18 / 1142.21 s -- reach,retrieve = (6.8 %, 6.8 %)                     
612 / 1000 in 726.82 / 1187.61 s -- reach,retrieve = (6.7 %, 6.7 %)                     
615 / 1000 in 728.06 / 1183.83 s -- reach,retrieve = (6.7 %, 6.7 %)                     
621 / 1000 in 734.31 / 1182.46 s -- reach,retrieve = (6.6 %, 6.6 %)                     
630 / 1000 in 744.26 / 1181.37 s -- reach,retrieve = (6.7 %, 6.7 %)                     
639 / 1000 in 753.28 / 1178.85 s -- reach,retrieve = (6.6 %, 6.6 %)                     
642 / 1000 in 754.62 / 1175.41 s -- reach,retrieve = (6.5 %, 6.5 %)                     
645 / 1000 in 758.54 / 1176.04 s -- reach,retrieve = (6.5 %, 6.5 %)                     
651 / 1000 in 795.55 / 1222.05 s -- reach,retrieve = (6.5 %, 6.5 %)                     
654 / 1000 in 798.5 / 1220.95 s -- reach,retrieve = (6.4 %, 6.4 %)                     
660 / 1000 in 800.69 / 1213.17 s -- reach,retrieve = (6.4 %, 6.4 %)                     
663 / 1000 in 803.33 /

513 / 1000 in 789.37 / 1538.73 s -- reach,retrieve = (7.2 %, 7.0 %)                     
516 / 1000 in 793.62 / 1538.02 s -- reach,retrieve = (7.2 %, 7.0 %)                     
522 / 1000 in 802.15 / 1536.69 s -- reach,retrieve = (7.1 %, 6.9 %)                     
531 / 1000 in 807.92 / 1521.5 s -- reach,retrieve = (7.0 %, 6.8 %)                      
534 / 1000 in 810.07 / 1516.98 s -- reach,retrieve = (6.9 %, 6.7 %)                     
537 / 1000 in 841.89 / 1567.77 s -- reach,retrieve = (6.9 %, 6.7 %)                     
549 / 1000 in 889.68 / 1620.55 s -- reach,retrieve = (6.9 %, 6.6 %)                     
552 / 1000 in 894.16 / 1619.86 s -- reach,retrieve = (6.9 %, 6.5 %)                     
558 / 1000 in 898.53 / 1610.26 s -- reach,retrieve = (7.0 %, 6.6 %)                     
567 / 1000 in 903.25 / 1593.04 s -- reach,retrieve = (6.9 %, 6.5 %)                     
570 / 1000 in 905.85 / 1589.21 s -- reach,retrieve = (6.8 %, 6.5 %)                     
576 / 1000 in 940.85 

984 / 1000 in 1640.84 / 1667.52 s -- reach,retrieve = (5.6 %, 5.3 %)                     
993 / 1000 in 1646.22 / 1657.83 s -- reach,retrieve = (5.5 %, 5.2 %)                     
996 / 1000 in 1648.74 / 1655.36 s -- reach,retrieve = (5.5 %, 5.2 %)                     
999 / 1000 in 1649.94 / 1651.59 s -- reach,retrieve = (5.5 %, 5.2 %)                     
1002 / 1000 in 1651.27 / 1647.98 s -- reach,retrieve = (5.5 %, 5.2 %)                     


============= Finished 1002 in 1651.27 s -- reach,retrieve = (5.5 %, 5.2 %) =================


In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()

reach_success_list = []
reach_time_list = []
retrieve_success_list = []
retrieve_time_list = []

for _ in range(1):
    ROBOT_DATA_ROOT = os.path.join(ALG_DATA_PATH, ROBOT_TYPE.name+"-failmore")
    try_mkdir(ROBOT_DATA_ROOT)

    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
    try_mkdir(DATASET_PATH)
    print("")
    print("DATASET_PATH: {}".format(DATASET_PATH))
    print("")

    GRASP_PATH = os.path.join(DATASET_PATH, "grasp")
    try_mkdir(GRASP_PATH)

    ARM_10_PATH = os.path.join(DATASET_PATH, "arm_10")
    try_mkdir(ARM_10_PATH)

    ARM_05_PATH = os.path.join(DATASET_PATH, "arm_05")
    try_mkdir(ARM_05_PATH)

    FULL_SCENE_PATH = os.path.join(DATASET_PATH, "full_scene")
    try_mkdir(FULL_SCENE_PATH)

    N_max_sample = 1000
    N_print = 5

    gtimer.tic("full_loop")

    i_s = 0
    i_print = 0
    reach_list = []
    retrieve_list = []
    while i_s < N_max_sample:
        ## add floor, ceiling
        floor = Floor(gscene, "floor")
        # ceiling = Ceiling(gscene, "ceiling")

        ## set workplane
        wp = random.choice(WORKPLANE_TYPES)(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

        ## add walls, poles, bars
        walls = redistribute_class(gscene, Wall, "wl", Nmax_wall, workplane_avoid=wp)
        poles = redistribute_class(gscene, Pole, "po", Nmax_pole, workplane_avoid=wp)
        bars = redistribute_class(gscene, Bar, "bar", Nmax_bar, workplane_avoid=wp)

        ## add object
        obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

        samples = []
        for obj in obj_list:
            SHOW_PERIOD = 0.01
            N_sample_single_obj = 3
            N_sample_single_obj_max = 3
            obj_pscene, handles = add_object(pscene, obj)
            mplan.update_gscene()
            initial_state = pscene.initialize_state(HOME_POSE)
            pscene.set_object_state(initial_state)
            from_state = initial_state.copy(pscene)
            to_node = ("grip0",)
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node, HOME_DICT)
            samples_obj = []
            for _ in range(N_sample_single_obj):
                to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
                
                pscene.set_object_state(from_state)
                
                Traj_self, LastQ, error, res, binding_list = \
                            checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                                    only_self_collision=True, timeout=TIMEOUT_SELF)
                if VISUALIZE and res:
                    gscene.show_motion(Traj_self)

                success_reach, success_retrieve = False, False
                time_reach, time_retrieve = TIMEOUT_REACH, TIMEOUT_RETRIEVE
                if res:
                    gtimer.tic("reach")
                    Traj_reach, LastQ, error, success_reach, binding_list = mplan.plan_transition(
                        from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, timeout=TIMEOUT_REACH)
                    time_reach = gtimer.toc("reach")
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
        #             print("reach: {}".format(success_reach))
                    Traj_retrieve = []
                    if success_reach:
                        if VISUALIZE:
                            gscene.show_motion(Traj_reach, period=SHOW_PERIOD)
                        for bd in binding_list:
                            pscene.rebind(bd, list2dict(LastQ, pscene.gscene.joint_names))
                        binding_state, state_param = pscene.get_object_state()
                        new_state = State(binding_state, state_param, list(LastQ), pscene)
                        end_state = new_state.copy(pscene)
                        end_state.Q = np.array(HOME_POSE)
                        gtimer.tic("retrieve")
                        Traj_retrieve, LastQ, error, success_retrieve, binding_list = mplan.plan_transition(
                            from_state=new_state, to_state=end_state, timeout=TIMEOUT_RETRIEVE)
                        time_retrieve = gtimer.toc("retrieve")
                        retrieve_success_list.append(success_retrieve)
                        retrieve_time_list.append(time_retrieve)
        #                 print("retrieve: {}".format(success_retrieve))
                        if success_retrieve and VISUALIZE:
                            gscene.show_motion(Traj_retrieve, period=SHOW_PERIOD)
                    samples_obj.append((obj.name, from_state, to_state, redundancy_dict, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
                    if len(samples_obj)>=N_sample_single_obj_max:
                        pscene.set_object_state(initial_state)
                        break
                else:
                    Traj_reach, Traj_retrieve = [], []
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
                    retrieve_success_list.append(success_retrieve)
                    retrieve_time_list.append(time_retrieve)
                    samples_obj.append((obj.name, from_state, to_state, redundancy_dict, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
#                 if success_retrieve and not res_col:
#                     raise(RuntimeError("Something's wrong: successful motion but collision checker failed"))
        #             print("no save?")
            samples = samples+samples_obj
            pscene.set_object_state(initial_state)

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)    

            gscene.update_markers_all()

        for sample in samples:
            obj_name, from_state, to_state, redundancy_dict, success_reach, success_retrieve, \
                Traj_reach, Traj_retrieve, time_reach, time_retrieve = sample
            obj = [obj for obj in obj_list if obj.name == obj_name][0]
            obj_pscene, handles = add_object(pscene, obj)
            pscene.set_object_state(from_state)
            binding_list, binding_ok = pscene.get_slack_bindings(from_state, to_state)
            assert binding_ok, "no available transition"
            assert len(binding_list) == 1, "multiple binding transition - only single allowed in training process"
            binding = binding_list[0]

            grasp_dict = {}
            arm_05_dict = {}
            arm_10_dict = {}
            full_scene_dict = {}
            ltc_effector.clear()
            ltc_arm_05.clear()
            ltc_arm_10.clear()
            ltc_full.clear()

            obj_name, ap_name, binder_name, binder_geometry_name = binding
            actor, obj = pscene.actor_dict[binder_name], pscene.subject_dict[obj_name]
            handle = obj.action_points_dict[ap_name]
            redundancy, Q_dict = redundancy_dict[obj_name], list2dict(from_state.Q, gscene.joint_names)
            redundancy = redundancy_dict[obj_name]
            point_add_handle, rpy_add_handle = calc_redundancy(redundancy[ap_name], handle)
            point_add_actor, rpy_add_actor = calc_redundancy(redundancy[binder_name], actor)
            
            T_handle_lh = np.matmul(handle.Toff_lh, SE3(Rot_rpy(rpy_add_handle), point_add_handle))
            T_actor_lh = np.matmul(actor.Toff_lh, SE3(Rot_rpy(rpy_add_actor), point_add_actor))
            T_loal = np.matmul(T_handle_lh, SE3_inv(T_actor_lh))
            actor_vertinfo_list, object_vertinfo_list, actor_Tinv_dict, object_Tinv_dict = \
                                                gcheck.get_grasping_vert_infos(actor, obj, T_loal, HOME_DICT)

            if all([ROBOT_NAME in lname for lname in actor_Tinv_dict.keys()]):
                tool_Tinv_dict = actor_Tinv_dict
                tool_vertinfo_list = actor_vertinfo_list
                target_vertinfo_list = object_vertinfo_list
                T_end_effector = T_loal
            elif all([ROBOT_NAME in lname for lname in object_Tinv_dict.keys()]):
                tool_Tinv_dict = object_Tinv_dict
                tool_vertinfo_list = object_vertinfo_list
                target_vertinfo_list = actor_vertinfo_list
                T_end_effector = SE3_inv(T_loal)
            else:
                raise("Invaild TOOL_LINK_BUNDLE")
            T_end_joint = T_end_effector
#             for lname in TOOL_LINK_BUNDLE:
#                 T_end_joint = np.matmul(T_end_joint, tool_Tinv_dict[lname])
#             raise

            r, th, h = cart2cyl(*T_end_effector[:3,3])
            Tref = SE3(Rot_axis(3, th), T_end_effector[:3,3])
            obj_names = [obj.geometry.name] + obj.geometry.children
            target_names= [item[0] for item in target_vertinfo_list if item[0] not in obj_names]
            tool_names = [item[0] for item in tool_vertinfo_list]

            ltc_effector.convert_vertices(tool_vertinfo_list, Tref=Tref)
            ltc_effector.convert_vertices(target_vertinfo_list, Tref=Tref)

            Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
            ltc_arm_05.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_full.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
            ltc_full.convert_vertices(actor_vertinfo_list, Tref=Tref_base)

            grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_end_effector, T_end_joint, Tref_base
            grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
            grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

            arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

            arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

            full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_end_effector, T_end_joint, Tref_base
            full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

            reach_list.append(success_reach)
            retrieve_list.append(success_retrieve)

            save_pickle(os.path.join(GRASP_PATH, "%06d.pkl"%i_s), grasp_dict)
            save_pickle(os.path.join(ARM_05_PATH, "%06d.pkl"%i_s), arm_05_dict)
            save_pickle(os.path.join(ARM_10_PATH, "%06d.pkl"%i_s), arm_10_dict)
            save_pickle(os.path.join(FULL_SCENE_PATH, "%06d.pkl"%i_s), full_scene_dict)

            i_s += 1

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)

            gscene.update_markers_all()
        if i_s > 0 :
            if i_s > i_print*N_print:
                i_print +=  1
                print_end = "\n"
            else:
                print_end = "\r"
            time_elapsed = gtimer.toc("full_loop")/1000
            print("{} / {} in {} / {} s -- reach,retrieve = ({} %, {} %)                     ".format(
                i_s, N_max_sample, round(time_elapsed, 2), round(time_elapsed/i_s*N_max_sample, 2), 
                round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)), end=print_end)


    print("")
    print("")
    print("============= Finished {} in {} s -- reach,retrieve = ({} %, {} %) =================".format(
        i_s, round(time_elapsed, 2), round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)))

## ====================== deprecated ============================

## prepare cells

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("prepare_reference_cells_ltc_full"):
    ltc_full.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_effector"):
    ltc_effector.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_05"):
    ltc_arm_05.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_10"):
    ltc_arm_10.prepare_reference_cells(gscene)
print(gtimer)

In [ ]:
gscene.show_motion(Traj_reach)

## visualize

In [ ]:
vistem_list = []
for col_idx in grasp_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in grasp_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in arm_10_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in arm_10_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in full_scene_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in full_scene_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## grasp conversion

In [ ]:
gtimer.reset()
with gtimer.block("convert_vertices"):
    ltc_effector.convert_vertices(actor_vertinfo_list, HOME_DICT, Tref=Tref)
    ltc_effector.convert_vertices(object_vertinfo_list, HOME_DICT, Tref=Tref)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_effector.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## scene conversion

In [ ]:
gtimer.reset()
with gtimer.block("ltc_arm_10_convert"):
    Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
    ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_arm_10.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## remove object

In [ ]:

pscene.remove_object(obj_pscene.oname)
for handle in handles:
    gscene.remove(handle)

In [ ]:
gscene.get_items_on_links(TOOL_LINK_BUNDLE[0]) + gscene.get_items_on_links(TOOL_LINK_BUNDLE[1])

In [ ]:
with gtimer.block("convert"):
    ltc_full.convert([gtem for gtem in gscene if gtem not in gtems_robot], crob.home_dict)

In [ ]:
print(gtimer)

In [ ]:
gscene.show_pose(crob.home_pose)

In [ ]:
# for coll_idxes in latticizer.coll_idx_dict.values():
#     for col_idx in coll_idxes:
#         gscene.copy_from(latticizer.cell_refs[col_idx])

## deprecated python ver

In [ ]:
from pkg.utils.gjk import make_point3, get_point_list, get_point_list_list, get_point_list_list_from_point_list
from pkg.utils.gjk import get_gjk_distance, get_gjk_distance_min, get_gjk_distance_all

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("get_cell_vertices"):
    gscene_ref = GeometryScene(gscene.urdf_content, gscene.urdf_path, gscene.joint_names, gscene.link_names, rviz=False)
    centers = get_centers(Nwdh, L_CELL, OFFSET_ZERO).reshape((-1,3))
    cell_refs = []
    for icell, center in enumerate(centers):
        cell_refs.append(gscene_ref.create_safe(GEOTYPE.BOX, str(icell), "base_link", dims=(L_CELL,)*3, 
                                                center=center, rpy=(0,0,0), color=(1,1,1,0.2), 
                                                display=True, collision=False, fixed=True))
    cell_vertices = get_cell_vertices(centers, L_CELL)
    cell_vertices_gjk = get_point_list_list(cell_vertices)
    center_vertices_gjk = get_point_list_list(np.expand_dims(centers, axis=-2))

In [ ]:
print(gtimer)

In [ ]:
gtimer.reset()
sqrt3 = np.sqrt(3)
LCmax = L_CELL*sqrt3/2
Qdict = crob.home_dict
with gtimer.block("full"):
    for gtem in gscene:
#         if gtem in gtems_robot or not gtem.collision:
#             continue
        with gtimer.block("centers"):
            Tgtem = SE3_inv(gtem.get_tf(Qdict))
            centers_loc = np.abs(np.matmul(centers, Tgtem[:3,:3].transpose())+Tgtem[:3,3])
        if gtem.gtype == GEOTYPE.BOX and np.sum(np.abs(gtem.rpy))<1e-5:
            with gtimer.block("right box"):
                dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - L_CELL, axis=-1)
                cell_idx_occupy = np.where(dist_list < 1e-3)[0]
                continue
        with gtimer.block("calc_center_dist"):
            dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - LCmax, axis=-1)
            idx_candi = np.where(dist_list<0)[0]
        with gtimer.block("calc_points"):
            gtem_verts, gtem_radius = gtem.get_vertice_radius_from(Qdict)
        with gtimer.block("get_point_gjk"):
            gtem_verts_gjk = get_point_list(gtem_verts)
        with gtimer.block("get_distance_gjk"):
            cell_idx_occupy = []
            for idx in idx_candi:
                cell_candi = cell_vertices_gjk[idx]
                center_v_candi = center_vertices_gjk[idx]
                if ((get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
                    cell_idx_occupy.append(idx)
#                 if ((get_gjk_distance(gtem_verts_gjk, center_v_candi) - gtem_radius - L_CELL/2 < 1e-4) or 
#                     (get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
#                     cell_idx_occupy.append(idx)
print(gtimer)